In [1]:
import copy
import pandas as pd


file = 'data.sql'
# 老师
teacher = pd.read_csv('teachers.csv',header = 0)
# print(teacher)
teacher['t_type'] = teacher['t_type'].replace([0,1],['教师','主任'])
# print(teacher.columns)

with open(file,"w") as f:
    for row in teacher.iterrows():
        f.write("INSERT INTO teacher VALUES ({0}, '{1}', '{2}', '{3}', '{4}', {5});".format(row[1][0],row[1][1],row[1][2],row[1][3],row[1][4],row[1][5])+'\n')



classes = pd.read_csv('classes.csv', header=0, )
with open(file,"a+") as f:
    for row in classes.iterrows():
        f.write("INSERT INTO class VALUES ({0}, {1}, '{2}');".format(row[1][0],row[1][1],row[1][2])+'\n')


# student
student = pd.read_csv('students.csv', header=0)
with open(file,"a+") as f:
    for row in student.iterrows():
        f.write("INSERT INTO student VALUES ({0}, {1}, '{2}', '{3}', to_date('2018-09-01','YYYY-MM-DD'),'计算机科学与技术', 123456 );".format(row[1][0],row[1][1],row[1][2], row[1][3])+'\n')



# courses
courses = pd.read_csv('courses.csv', header=0)
courses['is_compulsory'] = courses['is_compulsory'].replace([0, 1],['专业选修课', '专业必修课'])
with open(file,"a+") as f:
    for row in courses.iterrows():
        f.write("INSERT INTO course VALUES ({0}, '{1}', '{2}', '{3}', {4}, {5});".format(row[1][0],row[1][1], row[1][3], row[1][4], row[1][2], row[1][5])+'\n')

teacher_group = pd.read_csv('teacher_group.csv', header=0)
with open(file,"a+") as f:
    for row in teacher_group.iterrows():
        f.write("INSERT INTO teacher_group VALUES ({0}, '{1}');".format(row[1][0],row[1][1])+'\n')



# 将老师分配给班级
def split_teacher( teacher_group, classes):
    open_course = []
    with open('open.txt',"a+") as f:
        dic = {}
        for i in teacher_group:
            if i[0] not in dic.keys():
                dic[i[0]] = set([i[1]])
            else:
                dic[i[0]].add(i[1])
        # print(dic)

        for key in dic:
            tem_list = list(dic[key])

            if len(tem_list) == 1:
                for i in range(2): # 课程开的班次
                    for j in range(3): # 老师分到的课程
                        #print("新课程id{0}{1}, 任课老师id{2}, 班别id{3}, 课程原id{4}".format(key,i,tem_list[0], classes[3*i+j][0], key))
                        f.write("INSERT INTO open VALUES ({0}{1}, {2}, {3}, {4},{5});".format(key,i,tem_list[0], classes[3*i+j][0], key,0)+'\n')
                        open_course.append([key*10+i,tem_list[0], classes[3*i+j][0], key])
            elif len(tem_list) == 2:
                for i in range(2):
                    for j in range(3):
                        #print("新课程id{0}{1}, 任课老师id{2}, 班别id{3}, 课程原id{4}".format(key,i,tem_list[i], classes[3*i+j][0], key))
                        f.write("INSERT INTO open VALUES ({0}{1}, {2}, {3}, {4},{5});".format(key,i,tem_list[i], classes[3*i+j][0], key,0)+'\n')
                        open_course.append([key*10+i,tem_list[i], classes[3*i+j][0], key])


            elif len(tem_list) == 3:
                for i in range(3):
                    for j in range(2):
                        #print("新课程id{0}{1}, 任课老师id{2}, 班别id{3}, 课程原id{4}".format(key,i,tem_list[i], classes[2*i+j][0], key))
                        f.write("INSERT INTO open VALUES ({0}{1}, {2}, {3}, {4},{5});".format(key,i,tem_list[i], classes[2*i+j][0], key,0)+'\n')
                        open_course.append([key*10+i,tem_list[i],  classes[2*i+j][0], key])

            elif len(tem_list) == 4:
                for i in range(2):
                    for j in range(2):
                        #print("新课程id{0}{1}, 任课老师id{2}, 班别id{3}, 课程原id{4}".format(key,i,tem_list[i], classes[2*i+j][0], key))
                        f.write("INSERT INTO open VALUES ({0}{1}, {2}, {3}, {4},{5});".format(key,i,tem_list[i], classes[2*i+j][0], key,0)+'\n')
                        open_course.append([key*10+i,tem_list[i], classes[2*i+j][0], key])

                #print("新课程id{0}{1}, 任课老师id{2}, 班别id{3}, 课程原id{4}".format(key,2,tem_list[2], classes[4][0], key))
                #print("新课程id{0}{1}, 任课老师id{2}, 班别id{3}, 课程原id{4}".format(key,3,tem_list[3], classes[5][0], key))
                f.write("INSERT INTO open VALUES ({0}{1}, {2}, {3}, {4},{5});".format(key,2,tem_list[2], classes[4][0], key,0)+'\n')
                open_course.append([key*10+2,tem_list[2], classes[4][0], key])
                f.write("INSERT INTO open VALUES ({0}{1}, {2}, {3}, {4},{5});".format(key,3,tem_list[3], classes[5][0], key,0)+'\n')
                open_course.append([key*10+3,tem_list[3], classes[5][0], key])

            #print(key, dic[key],len(dic[key]))
    return open_course


open_course = split_teacher(teacher_group.values, classes.values)



In [2]:
import random

def is_teacher(x, t_list):
    if len(t_list) == 0:
        return False
    for t in t_list:
        if x in t:
            #print('冲突')
            return True
    return False

def get_class(open_course, courses):
    dic = {}
    for i in open_course:
        if i[0] not in dic.keys():
            dic[i[0]] = set([i[1]])
        else:
            dic[i[0]].add(i[1])
    new_open = []
    for key in dic:
        new_open.append([key]+list(dic[key]))

    dic = {}  # 每种课程的课时
    for i in courses:
        if i[0] not in dic.keys():
            dic[i[0]] = set([i[-1]])
        else:
            dic[i[0]].add(i[-1])
    # 在课程便添加课时
    for i in new_open:
        i.extend(list(dic[int(i[0]/10)]))
    # print(new_open)
    return new_open

def is_student(cour, schedule, dic):
    if len(schedule) ==0:
        return False

    cla = list(dic[cour])  # 要安排的课程的班级
    for i in schedule:
        cla1 = list(dic[i[0]])
        for j in cla:
            if j in cla1:
                return True
    return False

def cal_week(x):
    if x == 48:
        return '1-16周'
    elif x == 32:
        return '1-16周'
    elif x == 16:
        return '1-8周'
    elif x == 8:
        return '1-4周'


def paike(open_course, courses):
    result = []
    new_open = get_class(open_course, courses)
    dict_cour = {1:'星期一 1-2节', 2:'星期一 3-4节', 3:'星期一 5-6节', 4:'星期一 7-8节',
                 5:'星期二 1-2节', 6:'星期二 3-4节', 7:'星期二 5-6节', 8:'星期二 7-8节',
                 9:'星期三 1-2节', 10:'星期三 3-4节', 11:'星期三 5-6节', 12:'星期三 7-8节',
                 13:'星期四 1-2节', 14:'星期四 3-4节', 15:'星期四 5-6节', 16:'星期四 7-8节',
                 17:'星期五 1-2节', 18:'星期五 3-4节', 19:'星期五 5-6节', 20:'星期五 7-8节'}

    dic_night = [4,8,12,16,20];
    dic_night1 = {4:'星期一 7-8节', 8:'星期二 7-8节', 12:'星期三 7-8节', 16:'星期四 7-8节', 20:'星期五 7-8节'}

    schedule = dict([(k, []) for k in range(1,21)])

    dic = {}
    for i in open_course:
        if i[0] not in dic.keys():
            dic[i[0]] = set([i[2]])
        else:
            dic[i[0]].add(i[2])

    print(len(new_open))
    for cour in new_open:
        if cour[-1] == 48: # 48课时
            temp = random.randint(0, len(dic_night)-1)
            # cour[1]老师，schedule[dic_night[temp]]当天晚上已经安排了的课程
            # 课程， 当天晚上的课，
            while(is_teacher(cour[1], schedule[dic_night[temp]])|is_student(cour[0], schedule[dic_night[temp]], dic)):
                temp = random.randint(0, len(dic_night)-1)

            schedule[dic_night[temp]].append([cour[0], cour[1]])
            result.append([cour[0], cour[1], cal_week(cour[-1]), dic_night1[dic_night[temp]], dic[cour[0]]])
            print([cour[0], cour[1], cour[-1]],cal_week(cour[-1]), dic_night1[dic_night[temp]],dic[cour[0]]) # dic_night[temp],

        else:
            temp = random.randint(1, len(schedule))
            # cour[1]老师，schedule[dic_night[temp]]当天晚上已经安排了的课程
            # 课程， 当天晚上的课，
            # print(temp)
            # print(schedule[temp])
            while(is_teacher(cour[1], schedule[temp])|is_student(cour[0], schedule[temp], dic)):
                temp = random.randint(1, len(schedule))

            schedule[temp].append([cour[0], cour[1]])
            result.append([cour[0], cour[1], cal_week(cour[-1]), dict_cour[temp],dic[cour[0]]])

            print( [cour[0], cour[1], cour[-1]],cal_week(cour[-1]), dict_cour[temp],dic[cour[0]])# schedule[temp],

    # print(schedule)
    # 已知每节课的课时，
    return result

open_course = split_teacher(teacher_group.values, classes.values)

result = paike(open_course, courses.values)


26
[1806000260, 101048, 32] 1-16周 星期五 1-2节 {18061001, 18061002, 18061003}
[1806000261, 101048, 32] 1-16周 星期三 1-2节 {18061004, 18061005, 18061006}
[1806000270, 101048, 16] 1-8周 星期一 5-6节 {18061001, 18061002, 18061003}
[1806000271, 101048, 16] 1-8周 星期四 1-2节 {18061004, 18061005, 18061006}
[1806000170, 103168, 32] 1-16周 星期五 7-8节 {18061001, 18061002}
[1806000171, 110009, 32] 1-16周 星期三 5-6节 {18061003, 18061004}
[1806000172, 104087, 32] 1-16周 星期五 1-2节 {18061005, 18061006}
[1806000180, 103168, 16] 1-8周 星期四 5-6节 {18061001, 18061002}
[1806000181, 103325, 16] 1-8周 星期二 3-4节 {18061003, 18061004}
[1806000182, 104087, 16] 1-8周 星期四 7-8节 {18061005, 18061006}
[1806100010, 204896, 48] 1-16周 星期二 7-8节 {18061001, 18061002, 18061003}
[1806100011, 110001, 48] 1-16周 星期三 7-8节 {18061004, 18061005, 18061006}
[1806100020, 204896, 32] 1-16周 星期一 3-4节 {18061001, 18061002, 18061003}
[1806100021, 110001, 32] 1-16周 星期三 3-4节 {18061004, 18061005, 18061006}
[1806000190, 110002, 32] 1-16周 星期五 5-6节 {18061001, 18061002, 1806100

In [3]:

print(len(result))
# file
with open(file,"a+") as f:
    for i in result:
        for j in i[-1]:
            f.write("INSERT INTO open VALUES ({0}, {1}, {2}, {3}, '{4}','{5}',{6});".format(i[0],i[1],j, int(i[0]/10), i[3], i[2],0)+'\n')

26


In [4]:
# 每个同学安排上必修课
with open(file,"a+") as f:
    for res in result:
        for cou in courses.values:
            if (int(res[0]/10)==cou[0])&(cou[3]=='专业必修课'):
                for clas in res[-1]: # r是
                    for stu in student.values:
                        if stu[1]==clas:
                            f.write("INSERT INTO sc_choose VALUES ({0}, {1}, {2}, {3}, {4},{5});"
                                  .format(stu[0],res[0],cou[0] ,0, 0, 0)+'\n')




In [5]:
print(courses.values)

[[180600001 '计算机导论' 1.0 '专业必修课' '考试' 16]
 [180600002 '程序设计基础' 3.0 '专业必修课' '考试' 48]
 [180600003 '程序设计基础实验' 1.0 '专业必修课' '考查' 32]
 [180600004 '离散数学' 4.0 '专业必修课' '考试' 64]
 [180600007 '数据结构' 3.0 '专业必修课' '考试' 48]
 [180600008 '数据结构实验' 0.5 '专业必修课' '考查' 16]
 [180600009 '计算机组成与系统结构' 3.0 '专业必修课' '考试' 48]
 [180600010 '计算机组成与系统结构实验' 0.5 '专业必修课' '考查' 16]
 [180600011 '计算机网络' 3.0 '专业必修课' '考试' 48]
 [180600012 '计算机网络实验' 0.5 '专业必修课' '考查' 16]
 [180600013 '数据库原理' 3.0 '专业必修课' '考试' 48]
 [180600014 '数据库原理实验' 0.5 '专业必修课' '考查' 16]
 [180600015 '操作系统' 3.0 '专业必修课' '考试' 48]
 [180600016 '操作系统实验' 0.5 '专业必修课' '考查' 16]
 [180600017 '编译原理' 2.0 '专业必修课' '考试' 32]
 [180600018 '编译原理实验' 0.5 '专业必修课' '考查' 16]
 [180600080 '机器学习与数据挖掘' 3.0 '专业必修课' '考试' 48]
 [180600081 '机器学习与数据挖掘实验' 0.5 '专业必修课' '考查' 16]
 [180610001 '人工智能原理' 3.0 '专业必修课' '考试' 48]
 [180610002 '人工智能原理实验' 1.0 '专业必修课' '考查' 32]
 [180600030 '计算机视觉与模式识别' 3.0 '专业必修课' '考试' 48]
 [180600031 '计算机视觉与模式识别实验' 1.0 '专业必修课' '考查' 32]
 [180600102 '新生研讨课' 0.5 '专业选修课' '考查' 8]
 [180600005 '